In [1]:
# load
import pandas as pd
import lightgbm

data = pd.read_csv("X_train.csv", index_col=0)
data["mark"] = pd.read_csv("y_train.csv", index_col=0)["mark"]

stud_info = pd.read_csv("studs_info.csv", index_col=False)

X_validation = pd.read_csv("X_test.csv", index_col=0)

In [2]:
# rename columns
field_map = {
    "STD_ID": "stud",
    "НАПРАВЛЕНИЕ": "profession",
    "ГОД": "year",
    "АТТЕСТАЦИЯ": "exam_type",
    "ДИСЦИПЛИНА": "discipline",
    "КУРС": "course",
    "СЕМЕСТР": "semester",

    "   number": "number",
    "Пол": "sex",
    "Статус": "state",
    "Дата выпуска": "release_date",
    "Категория обучения": "category",
    "Форма обучения": "study_kind",
    "Шифр": "cipher",
    "направление (специальность)": "speciality",
    "   ": "what?",
    "Образование": "lvl_of_education",
    "Дата выдачи": "issue_date",
    "Что именно закончил": "education",
}

data.rename(columns=field_map, inplace=True)
X_validation.rename(columns=field_map, inplace=True)
stud_info.rename(columns=field_map, inplace=True)

In [3]:
%%capture
# clean up
# for some student: year == course + const
# for some student: course == ceil(semester / 2)
# therefore they are noise
fields = ["year", "course"]
data.drop(fields, axis=1, inplace=True)
X_validation.drop(fields, axis=1, inplace=True)

# all nulls and not present in data / validation
stud_info.drop(stud_info[stud_info["stud"] == 92222].index, inplace=True)

In [4]:
stud_info

,stud,number,sex,state,release_date,category,study_kind,cipher,speciality,what?,lvl_of_education,issue_date,education
0,99136,13951.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2629.0,СР,2013-06-26 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
1,99112,13998.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,8240.0,СР,2012-06-23 00:00:00,"МОУ СОШ N 2, ПЕСТОВО"
2,99109,13999.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2314.0,СР,2013-06-25 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
3,99125,13953.0,Ж,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2345.0,СР,2013-06-23 00:00:00,"Гимназия ""ГАРМОНИЯ"""
4,100676,13950.0,М,ВЫП,2019-08-31,БП,Д,31.05.01,Лечебное дело,2650.0,СР,2012-06-29 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5461,100678,978.0,Ж,СТ,NaN,БП,Д,31.05.01,Лечебное дело,2388.0,СР,2013-06-21 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
5462,100663,1021.0,Ж,СТ,NaN,ДОГ,Д,31.05.01,Лечебное дело,2554.0,СР,2013-06-27 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
5463,191630,6447.0,Ж,СТ,NaN,ДОГ,Д,31.05.01,Лечебное дело,6692.0,СР,2014-05-31 00:00:00,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
5464,191625,3765.0,Ж,СТ,NaN,ДОГ,Д,31.05.01,Лечебное дело,NaN,NaN,NaN,NaN


In [5]:
%%capture
stud_info.fillna({"lvl_of_education": "НЕТ"})

# data = data.merge(stud_info, how="left", on="stud")
# X_validation = X_validation.merge(stud_info, how="left", on="stud")
# data["stud"].map(stud_info.groupby("stud"))

In [6]:
lvl_of_education_order = [
    "НЕТ",  # 190  Нет данных
    "ОО",   # 160  Начальное общее образование
    "ОСН",  # 32   Основное общее образование
    "НС",   # 14   Незаконченное среднее образование
    "СР",   # 4101 Среднее общее образование
    "НВ",   # 2    Незаконченное высшее образование
    "НП",   # 50   Начальное/Незаконченное? профессиональное образование
    "СП",   # 916  Среднее профессиональное образование
]

# ОО - начальное общее образование
# ОСН - основное общее образование
# СР - среднее общее образование
# НС - незаконченное среднее
# НВ - незаконченное высшее
# НП - незаконченное профессиональное
# СП - среднее профессиональное


# Начальное (общее) образование                                                 Начальное общее образование
# Основное (общее) образование                                                  Основное общее образование
# Среднее (полное) общее образование                                            Среднее общее образование
# Начальное профессиональное образование                                        Среднее профессиональное образование по программам подготовки квалифицированных рабочих (служащих)
# Среднее профессиональное образование                                          Среднее профессиональное образование по программам подготовки специалистов среднего звена
# Высшее профессиональное образование - бакалавриат                             Высшее образование - бакалавриат
# Высшее профессиональное образование - подготовка специалиста или магистратура Высшее образование - специалитет или магистратура
# Послевузовское профессиональное образование в аспирантуре (адъюнктуре)        Высшее образование - подготовка кадров высшей квалификации по программам подготовки научно-педагогических кадров в аспирантуре (адъюнктуре)
# Послевузовское профессиональное образование в ординатуре                      Высшее образование - подготовка кадров высшей квалификации по программам ординатуры
# Послевузовское профессиональное образование в форме ассистентуры-стажировки   Высшее образование - подготовка кадров высшей квалификации по программам ассистентуры-стажировки

# set(stud_info["lvl_of_education"])
# stud_info[stud_info["lvl_of_education"] == 'СП']
# stud_info[pd.isnull(stud_info["study_kind"])]
# set(stud_info["category"])
stud_info[pd.isnull(stud_info["lvl_of_education"])]

,stud,number,sex,state,release_date,category,study_kind,cipher,speciality,what?,lvl_of_education,issue_date,education
113,170408,2273.0,Ж,СТ,NaN,ДОГ,З,38.03.01,Экономика,NaN,NaN,NaN,NaN
137,182351,3914.0,М,СТ,NaN,ДОГ,З,35.03.04,Агрономия,NaN,NaN,NaN,NaN
151,184634,10128.0,Ж,ВЫП,2020-01-31,ДОГ,З,40.04.01,Юриспруденция,NaN,NaN,NaN,NaN
191,170169,12801.0,М,ВЫП,2020-01-31,ДОГ,З,38.04.02,Менеджмент,NaN,NaN,NaN,NaN
198,170302,10166.0,Ж,ВЫП,2020-01-31,ДОГ,З,40.04.01,Юриспруденция,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5441,192577,6545.0,М,СТ,NaN,ДОГ,Д,31.05.01,Лечебное дело,NaN,NaN,NaN,NaN
5442,191636,5389.0,Ж,СТ,NaN,ДОГ,Д,31.05.01,Лечебное дело,NaN,NaN,NaN,NaN
5443,191638,5422.0,М,СТ,NaN,ДОГ,Д,31.05.01,Лечебное дело,NaN,NaN,NaN,NaN
5464,191625,3765.0,Ж,СТ,NaN,ДОГ,Д,31.05.01,Лечебное дело,NaN,NaN,NaN,NaN


In [7]:
data

,stud,profession,semester,exam_type,discipline,mark
51081,175711,38.03.02 Менеджмент,4.0,Экзамен,Маркетинг,5
72921,100647,31.05.01 Лечебное дело,5.0,Экзамен,"Пропедевтика внутренних болезней, лучевая диаг...",5
96438,199311,31.05.03 Стоматология,1.0,Экзамен,Химия,5
273044,162002,38.03.02 Менеджмент,3.0,Экзамен,Статистика,4
371309,108384,31.05.01 Лечебное дело,9.0,Экзамен,Педиатрия,3
...,...,...,...,...,...,...
224856,176141,40.03.01 Юриспруденция,2.0,Курсовая работа,Конституционное право,3
260998,182794,31.05.01 Лечебное дело,4.0,Экзамен,Нормальная физиология,3
57245,162391,31.05.01 Лечебное дело,2.0,Дифференцированный зачет,Иностранный язык,4
218529,202479,38.03.01 Экономика,1.0,Дифференцированный зачет,Информатика,4


In [8]:
# set(data["exam_type"])
set(data["discipline"])

{'Архитектурно-дизайнерское проектирование: основы архитектурно-дизайнерского проектирования. Теория и методология',
 'Русский язык как иностранный',
 'Алгебра многочленов',
 'Методы и средства контроля параметров точности изделий',
 'Прикладной стохастический анализ',
 'Практика производственная: практика научно-педагогическая',
 'Практика по написанию программного кода с использованием языков программирования, определения и манипулирования данными',
 'Проектирование цифровых устройств на программируемых логических интегральных микросхемах и микроконтроллерах',
 'Экономический анализ деятельности предприятия',
 'Художественная обработка древесины и металла',
 'Сверхвысокочастотные устройства электронных средств',
 'Дискретная математика и алгебраические структуры',
 'Современные технологии защиты окружающей среды',
 'Методика поискового проектирования в архитектуре',
 'Математические методы в проектировании и производстве микроэлектронных средств',
 'Инфекционные болезни',
 'Физика ко

In [9]:
# data[data["exam_type"] == "Экзамен"].sort_values(by=["mark"]).groupby(["stud"]).count()

In [10]:
# encode labels
from sklearn import preprocessing

fields = "discipline", "profession", "exam_type" #, "sex"
le_s = [preprocessing.LabelEncoder() for _ in fields]
for field_name, le in zip(fields, le_s):
    le.fit(pd.concat([data[field_name], X_validation[field_name]]))

    data[field_name] = le.transform(data[field_name])
    X_validation[field_name] = le.transform(X_validation[field_name])

# 69.0 to 69
fields = "semester",# "number", "what?"
for field_name in fields:
    data[field_name] = data[field_name].astype(int)
    X_validation[field_name] = X_validation[field_name].astype(int)

# normalize
data["semester"] -= 1

In [11]:
# means
fields = "stud", "profession", "discipline"
for field_name in fields:
    mean_mark = data.groupby(field_name).mean()["mark"]

    mean_name = field_name + "_mean"
    data[mean_name] = data[field_name].map(mean_mark)
    X_validation[mean_name] = X_validation[field_name].map(mean_mark)

In [12]:
data

,stud,profession,semester,exam_type,discipline,mark,stud_mean,profession_mean,discipline_mean
51081,175711,36,3,4,408,5,4.181818,4.157895,4.326389
72921,100647,24,4,4,946,5,4.116279,4.205835,4.266444
96438,199311,25,0,4,1326,5,4.750000,4.119935,3.569940
273044,162002,36,2,4,1100,4,4.172414,4.157895,3.721519
371309,108384,24,8,4,713,3,3.542857,4.205835,4.333919
...,...,...,...,...,...,...,...,...,...
224856,176141,40,1,2,364,3,3.615385,3.951955,3.833866
260998,182794,24,3,4,556,3,4.052632,4.205835,3.693413
57245,162391,24,1,0,228,4,4.458333,4.205835,4.153555
218529,202479,35,0,0,238,4,4.187500,4.114599,3.984946


In [13]:
# data[(data["stud"] == 171886) ].sort_values(by=["semester"])
# data[(data["stud"] == 100647) ].sort_values(by=["semester"])
# data[(data["year"] != data["course"]) ]
# data[data["semester"] == 0 ]

In [14]:
# create dummy variables
columns = []#"exam_type"]#, "discipline", "profession"]
data = pd.get_dummies(data, columns=columns)
X_validation = pd.get_dummies(X_validation, columns=columns)

# remove unneeded data
fields = ["stud", "discipline", "profession"]
data.drop(fields, axis=1, inplace=True)
X_validation.drop(fields, axis=1, inplace=True)

In [15]:
data

,semester,exam_type,mark,stud_mean,profession_mean,discipline_mean
51081,3,4,5,4.181818,4.157895,4.326389
72921,4,4,5,4.116279,4.205835,4.266444
96438,0,4,5,4.750000,4.119935,3.569940
273044,2,4,4,4.172414,4.157895,3.721519
371309,8,4,3,3.542857,4.205835,4.333919
...,...,...,...,...,...,...
224856,1,2,3,3.615385,3.951955,3.833866
260998,3,4,3,4.052632,4.205835,3.693413
57245,1,0,4,4.458333,4.205835,4.153555
218529,0,0,4,4.187500,4.114599,3.984946


In [16]:
from sklearn.model_selection import train_test_split

marks = data.pop("mark")
X_train, X_test, y_train, y_test = train_test_split(data, marks, shuffle=True, test_size=0.2)

# создание и обучение модели на основе бустинга рещающих деревьев
lgb = lightgbm.LGBMRegressor(n_estimators=1000, verbose=1)

lgb.fit(X_train, y_train)


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003713 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 580
[LightGBM] [Info] Number of data points in the train set: 84046, number of used features: 5
[LightGBM] [Info] Start training from score 4.181651


LGBMRegressor(n_estimators=1000, verbose=1)

In [17]:
from sklearn.metrics import mean_absolute_error, r2_score

print(mean_absolute_error(y_test, lgb.predict(X_test)))
print(r2_score(y_test, lgb.predict(X_test)))

y_pred = pd.read_csv("sample_submission.csv", index_col=0)
y_pred["mark"] = lgb.predict(X_validation)

y_pred.to_csv("baseline_submission.csv")


0.3863174911902344
0.5586128168906108
